In [1]:
import warnings
import sys
import os
import pathlib
import torch 
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import HTML
import time

warnings.filterwarnings('ignore')
sys.path.insert(0, str(pathlib.Path.cwd()/"src"))
%load_ext autoreload
%autoreload 2


In [2]:
import loaders.goes
import fields.continuous

In [3]:
goes = loaders.goes.Data(band=8, start="2024-09-26 12:00", end="2024-09-26 14:00", time = "2024-09-27 02:00")



 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/270/12/OR_ABI-L2-CMIPC-M6C08_G16_s20242701201174_e20242701203547_c20242701204068.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/270/12/OR_ABI-L2-CMIPC-M6C08_G16_s20242701221174_e20242701223547_c20242701224067.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/270/12/OR_ABI-L2-CMIPC-M6C08_G16_s20242701211174_e20242701213547_c20242701214064.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/270/12/OR_ABI-L2-CMIPC-M6C08_G16_s20242701216174_e20242701218547_c20242701219057.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/270/12/OR_ABI-L2-CMIPC-M6C08_G16_s20242701206174_e20242701208547_c20242701209031.nc
 👮🏻‍♂️ File already exists. Do not overwrite: /home/yf297/data/noaa-goes16/ABI-L2-CMIPC/2024/270/12/OR_ABI-L2-CMIPC

In [52]:
dsf = goes.scalar_field(extent = (-86,-84.5,24,26.5))

In [53]:
fig = dsf.plot(step = 3, factor = 2,  gif = True)
HTML(fig.to_html5_video())

In [56]:
cvf = fields.continuous.VectorField()
cvf.train(dsf, step = 6, factor = 3, size=1000, epochs = 90)

moved to gpu
sampling from 3080 points
flow lr: 0.0004 - Epoch 1/90 — Likelihood: 0.4666 — outputscale: 1.0026 — lengthscales: 2.25, 0.65, 0.65 — noise_var: 0.1004
flow lr: 0.0005 - Epoch 2/90 — Likelihood: 0.4468 — outputscale: 1.0054 — lengthscales: 2.24, 0.65, 0.65 — noise_var: 0.1005
flow lr: 0.0007 - Epoch 3/90 — Likelihood: 0.4581 — outputscale: 1.0087 — lengthscales: 2.24, 0.65, 0.65 — noise_var: 0.1006
flow lr: 0.0009 - Epoch 4/90 — Likelihood: 0.4479 — outputscale: 1.0131 — lengthscales: 2.23, 0.65, 0.64 — noise_var: 0.1004
flow lr: 0.0012 - Epoch 5/90 — Likelihood: 0.4686 — outputscale: 1.0188 — lengthscales: 2.22, 0.65, 0.64 — noise_var: 0.1006
flow lr: 0.0016 - Epoch 6/90 — Likelihood: 0.4551 — outputscale: 1.0261 — lengthscales: 2.21, 0.65, 0.64 — noise_var: 0.1008
flow lr: 0.0020 - Epoch 7/90 — Likelihood: 0.4420 — outputscale: 1.0353 — lengthscales: 2.20, 0.65, 0.63 — noise_var: 0.1007
flow lr: 0.0025 - Epoch 8/90 — Likelihood: 0.4294 — outputscale: 1.0466 — lengthscales

In [58]:
fig = cvf.plot(dsf.coord_field, step = 3, gif = True, factor = 5, scale = 300)
HTML(fig.to_html5_video())

In [46]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [14]:
gc.collect()


2986

In [57]:
dsf.coord_field.T[::6]

tensor([[43356.1367],
        [45156.1367],
        [46956.1445],
        [48756.1445]])